In [9]:
import datacube
from datacube.utils import geometry

import geopandas as gpd
import pandas as pd

from otps import TimePoint
from otps import predict_tide

import sys
sys.path.append("../Scripts")
from dea_plotting import map_shapefile
from dea_dask import create_local_dask_cluster

In [3]:
%cd '/home/jovyan/dev/dea-notebooks/Claire/'
vector_file = "ITEMv2_tidalmodel.shp"

attribute_col = "ID"

/home/jovyan/dev/dea-notebooks/Claire


In [4]:
# Read in the polygon vector file
gdf_master = gpd.read_file(vector_file)

In [5]:
gdf_master

,ID,lon,lat,LOT,HOT,LMT,HMT,geometry
0,1,118.96,-19.95,-2.349,2.787,-3.727,3.760,"POLYGON ((-1332551.097 -2291457.870, -1381922...."
1,10,123.39,-16.40,-4.728,2.714,-5.249,4.979,"POLYGON ((-899215.110 -1800457.194, -940647.26..."
2,100,146.84,-40.80,-1.768,1.311,-1.797,1.555,"POLYGON ((1256425.099 -4632170.094, 1249169.07..."
3,101,113.14,-24.76,-0.391,0.689,-0.702,0.701,"POLYGON ((-1895991.182 -2872822.977, -1943498...."
4,102,115.44,-32.74,-0.352,0.431,-0.440,0.464,"POLYGON ((-1616091.638 -3501126.241, -1531356...."
...,...,...,...,...,...,...,...,...
301,95,153.67,-28.77,-0.668,1.160,-0.905,1.212,"POLYGON ((2057991.209 -3430975.153, 1978311.80..."
302,96,138.18,-34.49,-1.136,0.770,-1.417,1.685,"POLYGON ((643612.278 -3823252.405, 514804.636 ..."
303,97,152.53,-23.99,-0.860,1.503,-1.256,1.545,"POLYGON ((2092752.326 -2808779.537, 1992884.12..."
304,98,136.78,-36.33,-0.377,0.446,-0.524,0.655,"POLYGON ((366164.538 -3948364.530, 450975.489 ..."


In [6]:
# Map the shapefiles from imported vector set
roi = map_shapefile(gdf_master, attribute=attribute_col)

Map(center=[-27.015, 133.06499999999997], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_…

## Plan: 
For each polygon and tidepost:
- load NIDEM
- load tidal model
- for continental run, select uniform time-filter and time-range to model exposure from
- run the exp_model function
- generate new exposure dataset from pixel drilling exp_model results
- export exposure ds as geotiff

TODO: 
- explore whether alchemist would benefit this workflow
- determine optimal time-range (use the global (~19year?) tidal cycle average? Can I increase the modelling frequency at the high and low tide extrema?)
- I think that the best time_filter will probably be all_time then tailored workflows can explore seasonality, diurnal exposure etc.

In [10]:
dc = datacube.Datacube(app="intertidal_exposure")
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:41067 Dashboard: /user/cp/proxy/8787/status,Cluster Workers: 1 Cores: 2 Memory: 13.11 GB


In [30]:
def tidal_stats(
#                 ds, 
                start='1/1/1988',
                end='1/1/2019',
                tidepost_lat=None,
                tidepost_lon=None,
                plain_english=True, 
#                 plot=True,
                modelled_freq='20min',
                round_stats=3): 
    """
    Takes an xarray.Dataset and statistically compares the tides 
    modelled for each satellite observation against the full modelled 
    tidal range. This comparison can be used to evaluate whether the 
    tides observed by satellites (e.g. Landsat) are biased compared to 
    the natural tidal range (e.g. fail to observe either the highest or 
    lowest tides etc).    
       
    By default, the function models tides for the centroid of the 
    dataset, but a custom tidal modelling location can be specified 
    using `tidepost_lat` and `tidepost_lon`.
    
    Tides are modelled using the OTPS tidal modelling software based on
    the TPXO8 tidal model: http://volkov.oce.orst.edu/tides/tpxo8_atlas.html
    
    For more information about the tidal statistics computed by this 
    function, refer to Figure 8 in Bishop-Taylor et al. 2018:
    https://www.sciencedirect.com/science/article/pii/S0272771418308783#fig8
    
    Parameters
    ----------     
    ds : xarray.Dataset
        An xarray.Dataset object with x, y and time dimensions  
    tidepost_lat, tidepost_lon : float or int, optional
        Optional coordinates used to model tides. The default is None,
        which uses the centroid of the dataset as the tide modelling 
        location.
    plain_english : bool, optional
        An optional boolean indicating whether to print a plain english 
        version of the tidal statistics to the screen. Defaults to True.
    plot : bool, optional
        An optional boolean indicating whether to plot how satellite-
        observed tide heights compare against the full tidal range. 
        Defaults to True.
    modelled_freq : str, optional
        An optional string giving the frequency at which to model tides 
        when computing the full modelled tidal range. Defaults to '2h', 
        which computes a tide height for every two hours across the
        temporal extent of `ds`.        
    round_stats : int, optional
        The number of decimal places used to round the output statistics.
        Defaults to 3.
        
    Returns
    -------
    A pandas.Series object containing the following statistics:
    
        tidepost_lat: latitude used for modelling tide heights
        tidepost_lon: longitude used for modelling tide heights
        observed_min_m: minimum tide height observed by the satellite
        all_min_m: minimum tide height from full modelled tidal range
        observed_max_m: maximum tide height observed by the satellite
        all_max_m: maximum tide height from full modelled tidal range
        observed_range_m: tidal range observed by the satellite
        all_range_m: full modelled tidal range 
        spread_m: proportion of the full modelled tidal range observed 
                  by the satellite (see Bishop-Taylor et al. 2018)
        low_tide_offset: proportion of the lowest tides never observed
                  by the satellite (see Bishop-Taylor et al. 2018)
        high_tide_offset: proportion of the highest tides never observed
                  by the satellite (see Bishop-Taylor et al. 2018)
        observed_slope: slope of any relationship between observed tide 
                  heights and time
        all_slope: slope of any relationship between all modelled tide 
                  heights and time
        observed_pval: significance/p-value of any relationship between 
                  observed tide heights and time
        all_pval: significance/p-value of any relationship between 
                  all modelled tide heights and time
    
    """
    
#     # Model tides for each observation in the supplied xarray object
#     ds_tides, tidepost_lon, tidepost_lat = tidal_tag(ds,
#                                                      tidepost_lat=tidepost_lat,
#                                                      tidepost_lon=tidepost_lon,
#                                                      return_tideposts=True)

    # Generate range of times covering entire period of satellite record
    all_timerange = pd.date_range(start=start,
                                  end=end,
                                  freq=modelled_freq)
    all_datetimes = all_timerange.values.astype('M8[s]').astype('O').tolist()  

    # Use the tidal model to compute tide heights for each observation:  
    all_timepoints = [TimePoint(tidepost_lon, tidepost_lat, dt) 
                      for dt in all_datetimes]
    all_predictedtides = predict_tide(all_timepoints)   
    all_tideheights = [predictedtide.tide_m for predictedtide 
                        in all_predictedtides]

#     if plot:
        
#         # Create plot and add all time and observed tide data
#         fig, ax = plt.subplots(figsize=(10, 5))
#         ax.plot(all_timerange, all_tideheights, alpha=0.4)

    return all_timerange, all_tideheights

In [35]:
## Development cell - trying to enable time based filtering of modelled tides (e.g. day, night, season etc) AND
## maintain indexing of results for selected parts of the tidal cycle.

# ##### Build a list of DataArrays to concat

# ## Identify to timeslice of interest from `modelledtides`
# time_start = 90000
# time_end = 215000

# ## NEW: index by date instead. This may break when time_start or time_end do not appear in the filtered dataset
# ## Ideally: 18.61 year window (19 years), the global average astronomically forced lunar nodal cycle (e.g. Haigh et al., 2011)
# time_start = '1988-12-01 00:00:00'
# time_end = '1989-12-01 00:00:00'

def exp_model(time_start, time_end, modelledtides, time_filter, tidepost_lat, tidepost_lon):
    
    '''
    For each modelled timestep in modelledtides,
    mask NIDEM for the associated tideheight.
    Save the lists of timesteps and masked arrays
    and concatenate to join all arrays on the new
    dimension 'datetime'
    
    Time filtering (`time_filter`) options for exposure mapping routine are:
    all_time,
    day,
    night,
    wet,
    dry,
    summer,
    autumn,
    winter,
    spring,
    Jan,
    Feb,
    Mar,
    Apr,
    May,
    Jun,
    Jul,
    Aug,
    Sep,
    Oct,
    Nov,
    Dec
    
    Use one of the following strings to match your tidepost to set the appropriate `timezone`
    UTC       =  pytz.UTC ## Coordinated Universal Time
    UTC + 8   = 'Australia/West'

    UTC + 9.5 = 'Australia/North'
                'Australia/South'

    UTC + 10  = 'Australia/Queensland'
                'Australia/NSW'
                'Australia/Victoria'
                'Australia/Tasmania'
    '''
    ## Automate timezone selection based on location of tidepost
    ## State boundary coords from https://www.ga.gov.au/scientific-topics/national-location-information/dimensions/border-lengths
    if tidepost_lon <= 129.0:
        timezone = 'Australia/West'
    elif (tidepost_lon > 129.0) & (tidepost_lon <= 138.0) & (tidepost_lat < 26.0):
        timezone = 'Australia/North'
    elif (tidepost_lon > 129.0) & (tidepost_lon <= 141.0) & (tidepost_lat > 26.0):
        timezone = 'Australia/South'
    elif (tidepost_lon > 138.0) & (tidepost_lat < 26.0):
        timezone = 'Australia/Queensland'
    elif (tidepost_lon > 141.0) & (tidepost_lat >= 26.0):
        timezone = 'Australia/NSW'
    else:
        print('Unable to assign timezone from supplied tidepost coords (tidepost_lon and tidepost_lat)')
    print('Timezone set to', timezone)
    
    ## Convert time_start and time_end to datetime objects in local timezone
    time_start = pd.to_datetime(time_start, utc=True).tz_convert(timezone)
    time_end = pd.to_datetime(time_end, utc=True).tz_convert(timezone)
    
    ## Convert tz-naive to tz-aware (set to UTC to match modelled timestamp input)
    localtides = modelledtides.index.tz_localize(tz=pytz.UTC)

    ## Convert tz-aware index to local timezone
    localtides = localtides.tz_convert(timezone)
    
    ## Reset the index of modelledtides to reflect local times
    modelledtides['local_timerange'] = localtides
    modelledtides.set_index('local_timerange', inplace=True)
    modelledtides.drop('timerange', axis=1, inplace=True)
    
    ## Create a filteredtides dataframe for the filter of interest
    if time_filter == 'all_time':
        filteredtides = modelledtides
        
    elif time_filter == 'night':
        night = localtides.drop(localtides[(localtides.hour == 6) ## Daylight: arbitrarily between 6 am and 5.59 pm
                    |(localtides.hour == 7)
                    |(localtides.hour == 8)
                    |(localtides.hour == 9)
                    |(localtides.hour == 10)
                    |(localtides.hour == 11)
                    |(localtides.hour == 12)
                    |(localtides.hour == 13)
                    |(localtides.hour == 14)
                    |(localtides.hour == 15)
                    |(localtides.hour == 16)
                    |(localtides.hour == 17)
                    ])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in night.array)
        filteredtides['timerange'] = night
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'day':
        day = localtides.drop(localtides[(localtides.hour == 0) ## Nighttime: arbitrarily between 6 pm and 5.59 am
                    |(localtides.hour == 1)
                    |(localtides.hour == 2)
                    |(localtides.hour == 3)
                    |(localtides.hour == 4)
                    |(localtides.hour == 5)
                    |(localtides.hour == 18)
                    |(localtides.hour == 19)
                    |(localtides.hour == 20)
                    |(localtides.hour == 21)
                    |(localtides.hour == 22)
                    |(localtides.hour == 23)
                    ])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in day.array)
        filteredtides['timerange'] = day
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'dry':
        dry = localtides.drop(localtides[(localtides.month == 10) ## Wet season: Oct-Mar
                    |(localtides.month == 11)
                    |(localtides.month == 12)
                    |(localtides.month == 1)
                    |(localtides.month == 2)
                    |(localtides.month == 3)
                    ])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in dry.array)
        filteredtides['timerange'] = dry
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'wet':
        wet = localtides.drop(localtides[(localtides.month == 4) ## Dry season: Apr-Sep
                    |(localtides.month == 5)
                    |(localtides.month == 6)
                    |(localtides.month == 7)
                    |(localtides.month == 8)
                    |(localtides.month == 9)
                    ])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in wet.array)
        filteredtides['timerange'] = wet
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)

    elif time_filter == 'summer':
        summer = localtides.drop(localtides[localtides.quarter != 1])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in summer.array)
        filteredtides['timerange'] = summer
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'autumn':
        autumn = localtides.drop(localtides[localtides.quarter != 2])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in autumn.array)
        filteredtides['timerange'] = autumn
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'winter':
        autumn = localtides.drop(localtides[localtides.quarter != 3])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in winter.array)
        filteredtides['timerange'] = winter
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'spring':
        spring = localtides.drop(localtides[localtides.quarter != 4])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in spring.array)
        filteredtides['timerange'] = spring
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'Jan':
        Jan = localtides.drop(localtides[localtides.month != 1])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in Jan.array)
        filteredtides['timerange'] = Jan
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
    
    elif time_filter == 'Feb':
        Feb = localtides.drop(localtides[localtides.month != 2])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in Feb.array)
        filteredtides['timerange'] = Feb
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'Mar':
        Mar = localtides.drop(localtides[localtides.month != 3])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in Mar.array)
        filteredtides['timerange'] = Mar
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'Apr':
        Apr = localtides.drop(localtides[localtides.month != 4])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in Apr.array)
        filteredtides['timerange'] = Apr
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'May':
        May = localtides.drop(localtides[localtides.month != 5])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in May.array)
        filteredtides['timerange'] = May
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'Jun':
        Jun = localtides.drop(localtides[localtides.month != 6])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in Jun.array)
        filteredtides['timerange'] = Jun
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'Jul':
        Jul = localtides.drop(localtides[localtides.month != 7])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in Jul.array)
        filteredtides['timerange'] = Jul
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'Aug':
        Aug = localtides.drop(localtides[localtides.month != 8])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in Aug.array)
        filteredtides['timerange'] = Aug
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'Sep':
        Sep = localtides.drop(localtides[localtides.month != 9])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in Sep.array)
        filteredtides['timerange'] = Sep
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'Oct':
        Oct = localtides.drop(localtides[localtides.month != 10])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in Oct.array)
        filteredtides['timerange'] = Oct
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'Nov':
        Nov = localtides.drop(localtides[localtides.month != 11])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in Nov.array)
        filteredtides['timerange'] = Nov
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)
        
    elif time_filter == 'Dec':
        Dec = localtides.drop(localtides[localtides.month != 12])
        
        ## Filtering modelledtides by nominated timerange
        filteredtides = pd.DataFrame(modelledtides['tideheights'][timestep] for timestep in Dec.array)
        filteredtides['timerange'] = Dec
        filteredtides.set_index('timerange', inplace=True)
        filteredtides.rename(columns={0:"tideheights"}, inplace=True)

    else:
        print('''
        Please select one of the following `time_filter`s: 
        all_time, day, night, wet, dry, summer, autumn, winter, 
        spring, Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec
        ''')
        
        
        
    ## Boolean mask all modelled timesteps by tideheight relative to NIDEM:
    da_list = list(ds.nidem >= filteredtides['tideheights'][timestep] 
                   for timestep in filteredtides.loc[time_start:time_end].index) 
     
    index_list = list(timestep for timestep in filteredtides[time_start:time_end].index)
    
    ## Store modelled datetimes as a pandas index
    index = pd.Index(index_list, name='datetime')

    ## Concatenate the arrays on the new dimension 'datetime'
    exp_ds = xr.concat(da_list, index)
    
    return exp_ds


# ## TEMP: refresh the modelledtides var
# modelledtides = pd.read_pickle("tidepost_"+str(tp_y) + '_'+ str(tp_x) + "_modelledtides_20min.pkl")

## Run function to collect exposure dataset. Set time filter to one of the following:
'''all_time,
    day,
    night,
    wet,
    dry,
    summer,
    autumn,
    winter,
    spring,
    Jan,
    Feb,
    Mar,
    Apr,
    May,
    Jun,
    Jul,
    Aug,
    Sep,
    Oct,
    Nov,
    Dec'''

'all_time,\n    day,\n    night,\n    wet,\n    dry,\n    summer,\n    autumn,\n    winter,\n    spring,\n    Jan,\n    Feb,\n    Mar,\n    Apr,\n    May,\n    Jun,\n    Jul,\n    Aug,\n    Sep,\n    Oct,\n    Nov,\n    Dec'

In [38]:
## DO NOT RUN THIS CELL - TOOOOOOO BIG!
## Prototyping the continental workflow
'''-----------'''

# ## NEW: index by date instead. This may break when time_start or time_end do not appear in the filtered dataset
## Ideally: 18.61 year window (19 years), the global average astronomically forced lunar nodal cycle (e.g. Haigh et al., 2011)
time_start = '1988-12-01 00:00:00'
time_end = '1989-12-01 00:00:00'

query = {
    'product': ['nidem'],
    "dask_chunks": {"time": 1, "x": 1000, "y": 1000}
        }
'''--------------------'''
## Additional code from intertidal WIT workflow
    ## Data loading from polyons rather than xy coords

align = (0, 0)

# results = {}

# Loop through polygons in geodataframe and extract satellite data
for index, row in gdf_master.iterrows():

#     print(f"Feature: {index + 1}/{len(gdf)}")
#     print(gdf["BRD_HAB"].values[index])
#     print(str(index))
#     print(str(row))

#     if not (str(row[attribute_col]) in results.keys()):
#         results[str(row[attribute_col])] = {}

    # Extract the feature's geometry as a datacube geometry object
    geom = geometry.Geometry(geom=row.geometry, crs=gdf_master.crs)

    # Update the query to include our geopolygon
    query.update({"geopolygon": geom})

    ds = dc.load(**query)
    ds = ds.where(ds.nidem > ds.nidem.min())
    
    '''-------------------'''
    ## run tidal model for polygon
        ## tidepost coords for polygon
    tp_x = row.lon
    tp_y = row.lat
    
    ## Run tidal model function
    all_timerange, all_tideheights = tidal_stats(tidepost_lat = tp_y,
                                             tidepost_lon = tp_x)
    ## Save modelled tides as dataframe
        ## Firstly convert the dates to series
    all_timerange = all_timerange.to_series()
        ## Save dates and tideheights to pd.DataFrame
    modelledtides = pd.DataFrame(data={'timerange': all_timerange,
                       'tideheights': all_tideheights})
    '''-------------------'''
    ## Run the exp_model function to generate the exposure dataset
    exp_ds = exp_model(time_start, 
                       time_end, 
                       modelledtides, 
                       time_filter = 'dry', 
                       tidepost_lat = tp_y, 
                       tidepost_lon = tp_x) 
    

    ## Calculate the mean of the boolean across the datetime axis (equiv. to % True)
    pxpc = exp_ds.mean('datetime')

    ## Compute the array from dask
    pxpc = pxpc.compute() #%timeit

    ## Name the exposure time array
    pxpc = pxpc.rename('Exposure time (%)')

    ## Mask out non-intertidal areas
    pxpc = pxpc.where(pxpc > 0)
    
    ## Save exposure results
    pxpc.drop('time').to_netcdf("ID"
                   + row.ID
                   + "_tidepost_"
                   + str(tp_y) 
                   + '_'
                   + str(tp_x) 
                   + "_pxpc_for_modelledtides20min_" 
                   + str(time_start)
                   + '_to_'
                   + str(time_end)
                   + "dry_season.nc")
    # ## To save the NIDEM dataset to netcdf, need to remove the time dimension
    ds = ds.squeeze()
    ds = ds.reset_coords()
    ds.nidem.to_netcdf("ID" + row.ID +"_tidepost_"+str(tp_y) + '_'+ str(tp_x) +'nidem.nc') 
    
    ## Stop looping
    if index == 3:
        break

KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker


In [39]:
# Loop through polygons in geodataframe and extract satellite data
for index, row in gdf_master.iterrows():
    print(index, row, row.lon, row.lat)
    if index == 3:
        break



0 ID                                                          1
lon                                                    118.96
lat                                                    -19.95
LOT                                                    -2.349
HOT                                                     2.787
LMT                                                    -3.727
HMT                                                      3.76
geometry    POLYGON ((-1332551.097375435 -2291457.87024134...
Name: 0, dtype: object 118.96 -19.95
1 ID                                                         10
lon                                                    123.39
lat                                                     -16.4
LOT                                                    -4.728
HOT                                                     2.714
LMT                                                    -5.249
HMT                                                     4.979
geometry    POLYGON ((-899215